<a href="https://colab.research.google.com/github/vish109/Machine_Learning/blob/main/Face_Mask_Detection/Face_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
import torchvision.models as models
from torchsummary import summary
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from google.colab.patches import cv2_imshow
import os

In [ ]:
# total images for each label = 2994+108+97+64
list_1 = os.listdir('gdrive/MyDrive/Datasets/Face_Mask/Dataset/mask_weared_incorrect_1')
list_2 = os.listdir('gdrive/MyDrive/Datasets/Face_Mask/Dataset/with_mask_1')
list_3 = os.listdir('gdrive/MyDrive/Datasets/Face_Mask/Dataset/without_mask_1')

In [ ]:
img0 = cv2.imread(f'gdrive/MyDrive/Datasets/Face_Mask/Dataset/with_mask_1/{list_2[0]}')
print(img0.shape[2])

3


In [ ]:

class MyData(Dataset):
  def __init__(self,file_len,list1,list2,list3,transform):
    self.file_len = file_len
    self.transform = transform

  def __len__(self):
    return self.file_len

  def __getitem__(self,index):
    if(index<2994):
      img_path = f'gdrive/MyDrive/Datasets/Face_Mask/Dataset/mask_weared_incorrect/{index+1}.png'
      image = cv2.imread(img_path)
      y_label = torch.tensor([1, 0, 0])

    elif(index>=2994 and index<2994*2):
      k = index-2994
      img_path = f'gdrive/MyDrive/Datasets/Face_Mask/Dataset/with_mask/{k+1}.png'
      image = cv2.imread(img_path)
      y_label = torch.tensor([0, 1, 0])
    elif(index>=2994*2 and index<2994*3):
      k = index-2994*2
      img_path = f'gdrive/MyDrive/Datasets/Face_Mask/Dataset/without_mask/{k+1}.png'
      image = cv2.imread(img_path)
      y_label = torch.tensor([0, 0, 1])
    elif(index>=2994*3 and index<(2994*3+108)):
      i = 0
      img_path = f'gdrive/MyDrive/Datasets/Face_Mask/Dataset/mask_weared_incorrect_1/{list_1[i]}'
      image = cv2.imread(img_path)
      y_label = torch.tensor([1,0,0])
    elif(index>=(2994*3+108) and index<(2994*3+108+97)):
      i = 0
      img_path = f'gdrive/MyDrive/Datasets/Face_Mask/Dataset/with_mask_1/{list_2[i]}'
      image = cv2.imread(img_path)
      y_label = torch.tensor([0,1,0])
    else:
      i = 0
      img_path = f'gdrive/MyDrive/Datasets/Face_Mask/Dataset/without_mask_1/{list_3[i]}'
      image = cv2.imread(img_path)
      y_label = torch.tensor([0,0,1])


    if(self.transform):
      image = self.transform(image)
    
    
    
    return(image,y_label)

In [ ]:
T = transforms.Compose([
                        #transforms.ToPILImage(),
                        #transforms.ColorJitter(brightness=0.5),
                        #transforms.RandomGrayscale(p=0.3),
                        transforms.ToTensor(),
                        #transforms.Normalize(mean=[0.38434757,0.41674298,0.48780654],std=[2.55361557e-07,1.66943286e-03,1.00000000e-09])
                        ])
data = MyData(2994*3+108+97+64,list_1,list_2,list_3,transform=T)

In [ ]:
train_set,val_set = torch.utils.data.random_split(data,[7401,1850])
validation_set,test_set = torch.utils.data.random_split(val_set,[925,925])
# k = [i for i in train_set if type(i)==None]
# print(k)

In [ ]:
train_load = DataLoader(dataset=train_set,batch_size=16,shuffle=True,drop_last=True)
val_load = DataLoader(dataset=validation_set,batch_size=16,shuffle=True,drop_last=True)
test_load = DataLoader(dataset=test_set,batch_size=16,shuffle=True,drop_last=True)
print(test_load)

In [ ]:
# def get_mean_std(loader):
#   channels_sum,channels_squared_sum,num_batches=0,0,0
#   for data, _ in loader:
#     channels_sum+=torch.mean(data,dim=[0,2,3])
#     channels_squared_sum+=torch.mean(data**2,dim=[0,2,3])
#     num_batches+=1

#   mean = channels_sum/num_batches
#   std = (channels_squared_sum/num_batches-mean**2)**0.5

#   return [mean,std]
# mean_std = get_mean_std(train_load)
# print(mean_std)

In [ ]:
T = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.ColorJitter(brightness=0.5),
                        transforms.RandomGrayscale(p=0.3),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.3868, 0.4193, 0.4881],std=[0.2488, 0.2489, 0.2706])
                        ])
data = MyData(2994*3+108+97+64,list_1,list_2,list_3,transform=T)

In [ ]:
train_set,val_set = torch.utils.data.random_split(data,[7401,1850])
validation_set,test_set = torch.utils.data.random_split(val_set,[925,925])
train_load = DataLoader(dataset=train_set,batch_size=16,shuffle=True,drop_last=True)
val_load = DataLoader(dataset=validation_set,batch_size=16,shuffle=True,drop_last=True)
test_load = DataLoader(dataset=test_set,batch_size=16,shuffle=True,drop_last=True)
print(test_load)

In [ ]:
# in_channels = 3
# num_classes = 3
# num_epochs = 5
# Model = Net(in_channels=in_channels,num_classes=num_classes)
# summary(Model,(3,128,128))

In [ ]:
# x = torch.randn(2,3,128,128)
# print(Model(x).shape)

Model1 = models.resnet50(pretrained=True)
# print(summary(Model1,(3,299,299))) 
print(Model1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [ ]:
for params in Model1.parameters():
  params.requires_grad = False
Model1.fc = nn.Sequential(nn.Linear(in_features=2048,out_features=1000),nn.Dropout(p=0.4,inplace=True),nn.Linear(in_features=1000,out_features=3))


In [ ]:
in_channels = 3
num_classes = 3
num_epochs = 25

In [ ]:
cost = nn.CrossEntropyLoss()
optimizer = optim.Adam(Model1.parameters(),lr=0.001)
lambda1 = lambda epoch: 0.5 ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)



#train loop
i = 0
for epoch in range(num_epochs):
  epoch_loss = []
  correct = 0
  for data in train_load:
    x,y = data
    scores = Model1(x)
    loss = cost(scores,torch.argmax(y,dim=1))
    epoch_loss.append(loss)
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    correct += (torch.argmax(scores,dim=1) == torch.argmax(y,dim=1)).float().sum()
  scheduler.step()
  print(f"epoch: {epoch+1}/{num_epochs} | loss: {sum(epoch_loss)/len(train_set)} | accuracy: {correct/len(train_set)}")


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 1/25 | loss: 0.039064664393663406 | accuracy: 0.8002972602844238
epoch: 2/25 | loss: 0.020155565813183784 | accuracy: 0.8763680458068848
epoch: 3/25 | loss: 0.01631009578704834 | accuracy: 0.9046074748039246
epoch: 4/25 | loss: 0.014938835054636002 | accuracy: 0.9089311957359314
epoch: 5/25 | loss: 0.01362330000847578 | accuracy: 0.9163626432418823
epoch: 6/25 | loss: 0.014139777980744839 | accuracy: 0.9160923957824707
epoch: 7/25 | loss: 0.013286453671753407 | accuracy: 0.9202810525894165
epoch: 8/25 | loss: 0.013763436116278172 | accuracy: 0.9205513000488281
epoch: 9/25 | loss: 0.013238433748483658 | accuracy: 0.9244696497917175
epoch: 10/25 | loss: 0.013628602959215641 | accuracy: 0.9154168367385864
epoch: 11/25 | loss: 0.013898693025112152 | accuracy: 0.917443573474884
epoch: 12/25 | loss: 0.013552827760577202 | accuracy: 0.9171733260154724
epoch: 13/25 | loss: 0.013555126264691353 | accuracy: 0.9193352460861206
epoch: 14/25 | loss: 0.013474014587700367 | accuracy: 0.9183893

In [ ]:
def check_accuracy(loader,Model1):
  correct = 0
  samples = 0
  Model1.eval()
  with torch.no_grad():
    for x,y in loader:
      scores = Model1(x)
      predictions = scores
      for i in range(len(y)):
        if(torch.argmax(scores[i])==torch.argmax(y[i])):
          correct = correct+1
        samples = samples+1
  print(f'got {float(correct)/float(samples)}')
check_accuracy(val_load,Model1)

got 0.9451754385964912


In [ ]:
check_accuracy(test_load,Model1)

got 0.9473684210526315


In [ ]:
# torch.save(Model1.state_dict(), 'Model.pth.tar')

In [ ]:
torch.save(Model1,'Entire_Model.pth')